## Getting Started With Langchain

- **Simple LLM calls with streaming**
- **Dynamic prompt templates** (translation app)
- **Building chains** (story generator with analysis)

In [1]:
import langchain
import os
from dotenv import load_dotenv

In [2]:
load_dotenv()

True

In [3]:
os.environ[ "GROQ_API_KEY" ] = os.getenv( "GROQ_API_KEY" )

### Example 1:  Simple LLM Call with streaming

In [4]:
from langchain.chat_models import init_chat_model
from langchain_core.messages import HumanMessage, SystemMessage

In [5]:
model = init_chat_model("groq:llama-3.1-8b-instant");

### Create Messages

In [6]:
messages = [
    SystemMessage("You are a helpful AI assistant"),
    HumanMessage("what are the top 2 benefits of using langchain?")
]

### Invoke the Model

In [7]:
response  = model.invoke(messages)
response

AIMessage(content='LangChain is an open-source framework for building large language models. The top 2 benefits of using LangChain are:\n\n1. **Ease of Use and Integration**: LangChain provides a simple and intuitive API for building and integrating large language models into applications. It allows developers to easily load and use pre-trained models, as well as create and train custom models. This makes it easier for developers to focus on building their applications without dealing with the complexities of model management.\n\n2. **Flexibility and Customizability**: LangChain provides a highly flexible framework for building large language models. It allows developers to create custom pipelines, integrate multiple models, and add custom logic to their applications. This flexibility makes it possible to build highly specialized and tailored models that can perform a wide range of tasks.\n\nThese benefits make LangChain a powerful tool for developers looking to build and deploy large 

In [8]:
response.content

'LangChain is an open-source framework for building large language models. The top 2 benefits of using LangChain are:\n\n1. **Ease of Use and Integration**: LangChain provides a simple and intuitive API for building and integrating large language models into applications. It allows developers to easily load and use pre-trained models, as well as create and train custom models. This makes it easier for developers to focus on building their applications without dealing with the complexities of model management.\n\n2. **Flexibility and Customizability**: LangChain provides a highly flexible framework for building large language models. It allows developers to create custom pipelines, integrate multiple models, and add custom logic to their applications. This flexibility makes it possible to build highly specialized and tailored models that can perform a wide range of tasks.\n\nThese benefits make LangChain a powerful tool for developers looking to build and deploy large language models in

### Streaming example

In [13]:
for chunk in model.stream(messages):
    print(chunk.content,end="",flush=True)

LangChain is an open-source framework for building large language models (LLMs). Based on my knowledge cutoff in 2023, here are the top 2 benefits of using LangChain:

1. **Modular and Extensible Architecture**: LangChain's modular design allows developers to easily integrate and combine different components, such as LLMs, databases, and APIs, to create custom workflows and applications. This flexibility enables developers to build complex systems that can handle various tasks, from simple question-answering to more advanced use cases like conversational AI and content generation.

2. **High-Performance and Scalability**: LangChain is optimized for performance, making it suitable for large-scale applications that require processing high volumes of data and generating responses quickly. Its ability to scale horizontally and handle concurrent requests enables developers to build robust and efficient systems that can handle increased traffic and user engagement.

Please note that LangChai

### Dynamic Prompt Template

In [14]:
from langchain_core.prompts import ChatPromptTemplate

translation_template = ChatPromptTemplate.from_messages([
    ("system" , "you are a professional translator. Translate the following {text} from {source_language} to {target_language}."),
    ("user", "{text}")
])

prompt = translation_template.invoke({
    "source_language":"English",
    "target_language":"Spanish",
    "text":"Hello, how are you? and also what are you doing these days?"
})

In [15]:
prompt

ChatPromptValue(messages=[SystemMessage(content='you are a professional translator. Translate the following Hello, how are you? and also what are you doing these days? from English to Spanish.', additional_kwargs={}, response_metadata={}), HumanMessage(content='Hello, how are you? and also what are you doing these days?', additional_kwargs={}, response_metadata={})])

In [16]:
translated_model = model.invoke(prompt)
translated_model.content

'The translation of "Hello, how are you? and also what are you doing these days?" from English to Spanish is:\n\n"Hola, ¿cómo estás? Y también, ¿qué estás haciendo estos días?"\n\nHowever, a more common and natural way to phrase this in Spanish would be:\n\n"Hola, ¿cómo estás? ¿Y qué tal? ¿Qué has estado haciendo últimamente?"\n\nHere\'s a breakdown of the translation:\n\n- "Hello" is translated to "Hola"\n- "how are you" is translated to "¿cómo estás?"\n- "and also" is translated to "Y también" or "¿Y qué tal?"\n- "what are you doing" is translated to "¿qué estás haciendo"\n- "these days" is translated to "estos días" or "últimamente" (meaning "lately" or "recently")'

### Building the first chain

In [43]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda

def createStoryChain():
    # Template for story generation
    storyPrompt = ChatPromptTemplate.from_messages([
        ("system","you are creative storyteller. Write a short and engaging story based on the given theme, character and settings. Maintain the narrative style and rarly break the third wall. Make sure the story is interesting and dont copy the an existing story. Make it exciting as possible."),
        ("user","Theme: {theme}\n Main character: {character}\n Setting: {setting}")
    ])

    # Template for story analysis
    analysisPrompt = ChatPromptTemplate.from_messages([
        ("system","You are a litery story critic. Analyze the following story and give insights like symbolisms how and what is inspired from where like scriptures or folklores or epics from any country. give appropriate ratings based on points Ranging from 0-3 and also provide ways in which the story can be made better."),
        ("user","story: {story}")
    ])

    # Creating the forst chain
    storyChain = (
        storyPrompt | model | StrOutputParser()
    )

    def analyzeStory(storyOutput):
        return { "story" : storyOutput }
    # Creating an analysis chain
    analysisChain = (
        storyChain | RunnableLambda(analyzeStory) | analysisPrompt | model | StrOutputParser()
    )

    return analysisChain

In [44]:
chain = createStoryChain()
chain

ChatPromptTemplate(input_variables=['character', 'setting', 'theme'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='you are creative storyteller. Write a short and engaging story based on the given theme, character and settings. Maintain the narrative style and rarly break the third wall. Make sure the story is interesting and dont copy the an existing story. Make it exciting as possible.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['character', 'setting', 'theme'], input_types={}, partial_variables={}, template='Theme: {theme}\n Main character: {character}\n Setting: {setting}'), additional_kwargs={})])
| ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001E28F5E9550>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001E28F725990>, model_name='llama-3.1-8b-instant'

In [45]:
result = chain.invoke({
    "theme":"Political tension/ Power strugle",
    "character":"arjun, a student from todays world",
    "setting":"arjun is reincarnated into a world full of magicals creatures and the era can be similar to midiviel times were swords and arrows were used. the world consists of 3 continents"
})
print(result)

**Story Analysis:**

The provided story is a fantasy tale that explores the themes of power, politics, and self-discovery. It combines elements of mythology, folklore, and world-building to create a rich and immersive setting. The story draws inspiration from various sources, including:

1. **Mythology and Folklore:** The world of Eridoria, with its diverse continents and magical creatures, bears similarities to J.R.R. Tolkien's Middle-earth. The concept of reincarnation and the idea of a mystical realm are reminiscent of Buddhist and Hindu mythologies.
2. **Epic Literature:** The story's focus on politics, power struggles, and alliances is reminiscent of epic tales like _The Iliad_ and _The Odyssey_. The character of Kael, with his pride and sense of purpose, echoes the likes of Achilles from Greek mythology.
3. **Fantasy Literature:** The story's use of magic, secret societies, and a council of elders is similar to the works of authors like George R.R. Martin and Patrick Rothfuss.

*